# Exploring the Datasets
*Authors: Angelika Shastapalava, Excel Espina, David Hadaller, Sam Mundle*  

### What data we are using:  
1) The "Discovery" API is MTA's official developer resource to get real-time data from their NYC Bus Time service. You can get more information <a href="http://bustime.mta.info/wiki/Developers/Index">here</a>  
2) Kaggle's NYC Bus Data <a href="https://www.kaggle.com/stoney71/new-york-city-transport-statistics">here</a>

### Goals, or how are we using the data:
Our plan is to explore the following:  
> 1. Predicting the probaility, given that a passenger approaches a bus stop within at a particular time interval during the day, that a bus will be late.
>2. What environmental factors impact a buses schedule? What impact does time of day,
temperature, and weather have?
> 3. Based on a ~10 stops/lines how closely do the actual stop times reflect the posted bus
schedules and what is the distribution around the scheduled time that busses actually
arrive?


### Sections:
1) [Background Information](#Background-Information) 

2) [Loading the Datasets](#Loading-the-Datasets) 

3) [Cleaning the Data](#Cleaning-the-Data)  

4) [Visualizing the Data](#Visualizing-the-Data)

## Background Information

#### Background Reading for Goal 1
Doing some background reading, we came across two articles, one [by Jake Vanderplas](http://jakevdp.github.io/blog/2018/09/13/waiting-time-paradox/), developer of data science tools and director open software at the University of Washington's eScience Institute, and another [by Allen Downey](http://allendowney.blogspot.com/2015/08/the-inspection-paradox-is-everywhere.html), a Professor of Computer Science at Olin College and author of many data science texts. Both of their articles coincide with our research goals, but since they're similar, we'll focus mainly Allen Downey's work.  

In his article, Downey discusses the inspection paradox as applied to many different data sets, including train arrival times. What follows is a brief summary of the inspection paradox, which explains why most people can truthfully report that trains are overcrowded and late, despite the train operator's equally truthful observation that *most* trains aren't drastically late or crowded. Say that the mean wait time between busses is 10 minutes. Also assume that if train B is late, train A cannot step in to serve those waiting customers even if it's closer. So then, given that late trains, by definition, arrive after a long wait interval, then if we're choosing a random time to approach the patform, there's more of a chance that the train we board will be a late train. This is especially true if the time interval between trains features a high variance. Based on the scenario described, the waiting time paradox also makes the claim that ,when the average time between arrivals is $n$, customers experience an average wait interval experienced of $2n$.

Allen Downey tests this claim this by plotting the actual distribution of time between trains on the Red Line in Boston, and the biased distribution that would be observed by passengers of that line. His observations and a plot of that data are as follows:

> The average time between trains is 7.8 minutes, so we might expect the average wait time to be 3.8 minutes.  But the average of the biased distribution is 8.8 minutes, and the average wait time for passengers is 4.4 minutes, about 15% longer.
> 
> In this case the difference between the two distributions is not very big because the variance of the actual distribution is moderate.  When the actual distribution is long-tailed, the effect of the inspection paradox can be much bigger.


<img src="images/1-time-between-trains.png" alt="drawing" width="600"/>

Of course, as he explains in the above quote, Downey does not find the same trend as the idealized inspecition paradox--when the average time between arrivals is $n$, customers experience an average wait interval experienced of $2n$. Certainly, small variance he cites This is a likely reason for these observations, however, it's can also be true that passengers do not approach the train platform at a *uniformly* random time. Mass transit usage likely features some other kind of distribution (perhaps a bimodal distibution, since people commute to work in the morning and then back home in the evening.) 

For our purposes, creating a biased distribution to test whether the MTA bus data can be described by the inspection paradox would require two things:
1. Bus times deltas: that is, the intervals between different bus arrival times at a given stop. We need this to compute the actual ecdf (shown in dark blue.) 
2. Passenger Data: we need to know how many passengers boarded each bus that stopped at the particular stop in question. This data would be used to create the light blue, biased ecdf, which describes probability from the passenger's perspective.

This biased distribution would help us guage probability from the customer standpoint. We're not pointing at busses as they leave the station, assessing the probability as to whether they'll be late. Instead, we're guaging the probability that a customer, who makes his/her way to a stop, will board a late bus.

Once we have this biased distribution, we can find the impact of conditional probabilities due to various weather events (taken from external data) on average user wait time (the biased chart.) This leads us into goal 2.

### Our Data Analysis
The goal of our data analysis is to build on what Downey's work, only with the MTA bus times. We aim to find the distribution of time intervals between one bus leaving and the next bus coming (we call these "time deltas"). If these time intervals feature a great deal of variance, many passengers will experience the inspection paradox; i.e. if they approach a bus stop at a uniform, random time, even if the average (mean) bus arrives on time, most passengers will experience long wait times. 

After we've created an ecdf of the time deltas, we can later simulate the probability that a given passenger will experience a delay using montecarlo methods and/or add conditional probabilities to our analysis with weather data. We must simulate passenger behavior with montecarlo methods because, unlike the Red Line in Boston, the MTA only publishes ridership data as a year-end sum, not for individual bus rides.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime, math, glob

### Loading the Datasets 
We want to work with the Kaggle dataset so head over <a href="https://www.kaggle.com/stoney71/new-york-city-transport-statistics">here</a> and download the zip file. (A word of caution: the dataset is approx **5GB** when extracted!)

After you extract the data, we want to load a csv on our notebook.  

The `error_bad_lines=False` parameter fixes some formatting issues when we load in our dataset.

In [2]:
%%capture
df = pd.concat([pd.read_csv(f, error_bad_lines=False) for f in glob.glob('mta*.csv')], ignore_index = True)

## Cleaning the Data
For this EDA, we're only going to be looking at the **M100** bus with the route going to **Inwood 220 St Via Amsterdam Via Bway**. We are interested in the stop data in **W 125 St/St Nicholas Av**

In [3]:
df_m100 = df.loc[(df['PublishedLineName']== 'M100') & (df['DestinationName'] == 'INWOOD 220 ST via AMSTERDAM via BWAY'),]


We want to look at M100 buses that have reported ```at stop``` on ```ArrivalProximityText``` whose ```OriginName``` is the stop before **W 125 St/St Nicholas Av**, ```W 125 ST/FRED DOUGLASS BL```

In [4]:
# M100_FD = m100.loc[(m100['ArrivalProximityText'] == 'at stop') & (m100['NextStopPointName'] == 'W 125 ST/FRED DOUGLASS BL'),]
df_nick_m100 = df_m100.loc[(df_m100['ArrivalProximityText'] == 'at stop') & (df_m100['NextStopPointName'] == 'W 125 ST/ST NICHOLAS AV'),]


In [5]:
df_nick_m100.shape

(384, 17)

Now we have a dataframe of M100 buses that have stopped at Nicholas. 

Our next step would be to compare the expected arrival to scheduled.

~~Also, how about we limit our the scope to 7 days from Aug 5 to Aug 11.~~

In [6]:
# Changing obj to datetime 
df_nick_m100['RecordedAtTime'] = pd.to_datetime(df_nick_m100['RecordedAtTime'])

# Sort the time 
df_nick_m100 = df_nick_m100.sort_index()

# M100_FD[(M100_FD['RecordedAtTime'] > '2017-08-05 05:00:00') & 
#             (M100_FD['RecordedAtTime'] < '2017-08-11 21:00:00')]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Getting the Time Deltas between Buses

First changing the object types to datetime.

In [7]:
df_nick_m100['ExpectedArrivalTime'] = pd.to_datetime(df_nick_m100['ExpectedArrivalTime'])

Then calculate the delta

In [8]:
df_nick_m100['time_diff'] = df_nick_m100['ExpectedArrivalTime'].diff()

Converting to mins.

In [9]:
df_nick_m100['time_diff_mins'] = df_nick_m100['time_diff'].dt.total_seconds().div(60, fill_value=0).apply(math.ceil).astype(int)

In [10]:
df_nick_m100.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,time_diff,time_diff_mins
57746,2017-06-01 08:54:05,0.0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_8379,40.810948,-73.953024,W 125 ST/ST NICHOLAS AV,at stop,1.0,2017-06-01 08:54:33,08:54:39,NaT,0
279966,2017-06-02 07:15:59,0.0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_4345,40.810930,-73.952982,W 125 ST/ST NICHOLAS AV,at stop,5.0,2017-06-02 07:16:06,07:18:48,22:21:33,1342
340552,2017-06-02 11:06:12,0.0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_4330,40.810948,-73.953024,W 125 ST/ST NICHOLAS AV,at stop,1.0,2017-06-02 11:06:23,10:44:39,03:50:17,231
346454,2017-06-02 11:35:47,0.0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_8397,40.810915,-73.952947,W 125 ST/ST NICHOLAS AV,at stop,9.0,2017-06-02 11:35:54,11:34:39,00:29:31,30
437987,2017-06-02 17:55:44,0.0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_8385,40.810861,-73.952818,W 125 ST/ST NICHOLAS AV,at stop,21.0,2017-06-02 17:55:59,18:05:29,06:20:05,381


Now we save our progress to a CSV file.

In [11]:
df_nick_m100.to_csv('M100_4_month_W125_st.csv', encoding='utf-8', index=False)

PermissionError: [Errno 13] Permission denied: 'M100_4_month_W125_st.csv'

In [12]:
df.isnull().sum()

RecordedAtTime                     0
DirectionRef                  301238
PublishedLineName                  0
OriginName                    604571
OriginLat                     604571
OriginLong                    604571
DestinationName                    0
DestinationLat                350671
DestinationLong               350671
VehicleRef                         0
VehicleLocation.Latitude           0
VehicleLocation.Longitude          0
NextStopPointName             329079
ArrivalProximityText           28091
DistanceFromStop               28091
ExpectedArrivalTime          3976960
ScheduledArrivalTime         1027910
dtype: int64

In [13]:
df.shape

(26520716, 17)

In [14]:
df_nick_m100.isnull().sum()

RecordedAtTime                 0
DirectionRef                   0
PublishedLineName              0
OriginName                     5
OriginLat                      5
OriginLong                     5
DestinationName                0
DestinationLat                 0
DestinationLong                0
VehicleRef                     0
VehicleLocation.Latitude       0
VehicleLocation.Longitude      0
NextStopPointName              0
ArrivalProximityText           0
DistanceFromStop               0
ExpectedArrivalTime           85
ScheduledArrivalTime           5
time_diff                    152
time_diff_mins                 0
dtype: int64

In [15]:
df_nick_m100.shape

(384, 19)

We then import the csv as a dataframe, because my (David Hadaller) laptop cannot handle importing 'mta_1708.csv', a 1.5 GB file, as Excel did in the cells above.

In [16]:
M100_NICK = pd.read_csv('M100_month_W125_st.csv')

In [17]:
def ecdf(inputSeries):
    try:
        x = np.sort(inputSeries)
    except:
        print("Warning: Series Unsorted")
        x = inputSeries
    y = np.arange(1, len(x)+1) / len(x)
    _ = plt.plot(x, y, marker='.', linestyle='none')
    _ = plt.xlabel('Time Delta (minutes)')
    _ = plt.ylabel('ECDF')
    plt.margins(0.02) # Keeps data off plot edges
    plt.show()
    
def hist(inputSeries):
    plt.hist(inputSeries, bins=25, density=True)
    _ = plt.xlabel('Time Delta (minutes)')
    _ = plt.ylabel('PDF')
    plt.show()

The charts below represent the ecdf and pdf of the bus average "time deltas" or time intervals between the departure of of one bus and the arrival of the next, over the course of the month of August in 2017. Also included are some acompanying summary statistics. Notice that, apart from a single outlier, these measurements do form some sort of distribution. Parameter estimation by bootstrapping could help us validate some hypotheses. Once we have a proper theoretical distribution to model this data, we can then use montecarlo metods to simluate passenger experience. So, what we've done is to derive the actual cdf (the dark blue line in the chart on the background information section), which means generating the biased cdf (the light blue line in the same background section) remains.

In [ ]:
M100_NICK_Avg = M100_NICK[['RecordedAtTime','time_delta_mins']]
dates = M100_NICK_Avg['RecordedAtTime'].str.split(' ', 1, expand=True).drop([1],axis=1)
M100_NICK_Avg = pd.concat([dates,M100_NICK_Avg['time_delta_mins']],axis=1).rename(columns={0:'Date','time_delta_mins':'TimeDelta'})
M100_NICK_Avg = M100_NICK_Avg.groupby('Date').mean().dropna()
M100_NICK_Avg.head()

In [ ]:
 ecdf(M100_NICK_Avg['TimeDelta'])
hist(M100_NICK_Avg['TimeDelta'])
M100_NICK_Avg['TimeDelta'].describe()

Below, we have also begun to plot time delta behavior for an individual day. 

In [ ]:
M100_NICK_day1 = M100_NICK[['RecordedAtTime','time_delta_mins']]
dates = M100_NICK_day1['RecordedAtTime'].str.split(' ', 1, expand=True)
M100_NICK_day1 = pd.concat([dates,M100_NICK_day1['time_delta_mins']],axis=1).rename(columns={0:'Date',1:'RecordingTime','time_delta_mins':'TimeDelta'})
M100_NICK_day1 = M100_NICK_day1[M100_NICK_day1['Date']=='2017-08-01']
M100_NICK_day1 = M100_NICK_day1.drop_duplicates().drop(['Date'],axis=1).groupby('RecordingTime')
M100_NICK_day1.head()

In [ ]:
#Visualizing the Time Delta for one day's trips
#day1Times = M100_NICK_day1['TimeDelta']
#day1Times.head()
#ecdf(day1Times)
#hist(day1Times)
#day1Times.describe()